In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [52]:
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression

In [41]:
raw = pd.read_csv('Trading_Project_Data.csv', index_col = 0, parse_dates = True, header = [0, 1])
raw = raw.dropna()
raw

/var/folders/2j/h6y_zlpd5sq1fhr8v9460g600000gn/T/ipykernel_4151/2967121963.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  raw = pd.read_csv('Trading_Project_Data.csv', index_col = 0, parse_dates = True, header = [0, 1])


Price            Close                                                  \
Ticker            AAPL        AMZN          BA         CAT         CVX   
Date                                                                     
2024-01-22  192.944748  154.779999  214.929993  284.386902  136.604904   
2024-01-23  194.228455  156.020004  211.500000  284.544495  136.221420   
2024-01-24  193.551773  156.869995  214.130005  286.367065  138.934540   
2024-01-25  193.223389  157.750000  201.880005  296.307312  142.433807   
2024-01-26  191.481918  159.119995  205.470001  294.987213  142.980240   
...                ...         ...         ...         ...         ...   
2025-01-10  236.850006  218.940002  172.000000  349.717926  153.139999   
2025-01-13  234.399994  218.460007  170.570007  361.175934  155.350006   
2025-01-14  233.279999  217.759995  167.020004  370.212799  156.899994   
2025-01-15  237.869995  223.350006  166.199997  373.520691  158.330002   
2025-01-16  228.259995  220.660004  168.929993  379.159973  159.380005   

Price                                                                  ...  \
Ticker           GOOGL          GS         JNJ         JPM         KO  ...   
Date                                                                   ...   
2024-01-22  145.462692  376.831573  157.484207  166.308197  57.804554  ...   
2024-01-23  146.508896  371.764343  154.905838  165.213226  58.076252  ...   
2024-01-24  148.162903  370.426758  154.081940  166.689453  57.164120  ...   
2024-01-25  151.321442  373.648712  154.663498  169.074951  57.406708  ...   
2024-01-26  151.640289  368.854858  154.605347  168.429688  57.610481  ...   
...                ...         ...         ...         ...        ...  ...   
2025-01-10  192.039993  560.000000  142.059998  239.869995  61.070000  ...   
2025-01-13  191.009995  562.950012  144.470001  244.210007  61.650002  ...   
2025-01-14  189.660004  571.530029  144.750000  247.470001  62.040001  ...   
2025-01-15  195.550003  605.919983  144.970001  252.350006  61.770000  ...   
2025-01-16  192.910004  612.989990  147.770004  254.270004  62.250000  ...   

Price          Volume                                                   \
Ticker            MCD        MSFT         NKE         NVDA         PFE   
Date                                                                     
2024-01-22  3250200.0  27016900.0   9897400.0  452955000.0  41663900.0   
2024-01-23  1874000.0  20525900.0   8208000.0  294654000.0  33466700.0   
2024-01-24  3209700.0  24867000.0   7879300.0  560271000.0  42720700.0   
2024-01-25  3219600.0  21021200.0  10063700.0  482777000.0  47605500.0   
2024-01-26  4210500.0  17803300.0   9735800.0  390309000.0  47073800.0   
...               ...         ...         ...          ...         ...   
2025-01-10  3946000.0  20201100.0  15677400.0  207602500.0  32836900.0   
2025-01-13  3394300.0  17604800.0  14584300.0  204808900.0  28359600.0   
2025-01-14  2238900.0  16935900.0  16732900.0  195590500.0  25856900.0   
2025-01-15  2867800.0  19637800.0  13468100.0  185217300.0  35949700.0   
2025-01-16  3403700.0  15300000.0  13694600.0  209235600.0  29981100.0   

Price                                                                    
Ticker            SOFI         TSLA         UNH         WMT         XOM  
Date                                                                     
2024-01-22  69046700.0  117952500.0   5922900.0  13938300.0  19955900.0  
2024-01-23  45600700.0  106605900.0   2672900.0  12066600.0  15863400.0  
2024-01-24  41431000.0  123369900.0   2918800.0  22932900.0  17330600.0  
2024-01-25  49509400.0  198076800.0  14671000.0  16369500.0  22089500.0  
2024-01-26  72566000.0  107343200.0   6212200.0  15746400.0  20817200.0  
...                ...          ...         ...         ...         ...  
2025-01-10  33470100.0   62287300.0   5467500.0  18140900.0  19304500.0  
2025-01-13  32197100.0   67580500.0   6472500.0  18617100.0  17073400.0  
2025-01-14  3410080

In [26]:
dates = raw.index

# find index to date 2024-03-01
index = dates.get_loc('2024-03-01')

index

28

In [139]:
def models_fit(data, cols, models):

    for model in models:
        model.fit(data[cols], data['Direction'])

    return models

def models_predict(data, cols, models):

    preds = []
    for model in models:
        preds.append(model.predict(data[cols]))

    final_pred = vote_for_signal(preds)

    return final_pred    

def vote_for_signal(preds):

    final_pred = np.mean(preds)

    final_pred = np.where(final_pred >= 0, 1, -1)

    return final_pred

In [140]:
def get_signal(models, train_df, features, pred_feature = 'Direction'):

    models_fit(train_df, features, models)

    predict_data = train_df.iloc[[-1]].copy()

    pred = models_predict(predict_data[features], features, models)

    return pred
        

In [160]:
def get_investment_amount(signals, wealth, df_stocks):

    investemnt_amounts = len(df_stocks) * [wealth / len(df_stocks)]

    return investemnt_amounts

In [179]:
def trade(df_stocks, df_features, wealth = 50000):

    dates = df_features.index

    # find index to date 2024-03-01
    index = dates.get_loc('2024-03-01')

    # calculate the indices amount left
    indices_left = len(dates) - index

    # inital split data into training window and testing
    train_df = df_features[:index]

    for i in range(1, indices_left):

        signals = []
        invest_amounts = []
        strat = []
        returns = []

        for j in range(len(df_stocks)):

            # get all stocks ML specifications
            stock = df_stocks['name'][j]
            models = df_stocks['models'][j]
            features = df_stocks['features'][j]
            pred_feature = df_stocks['pred_feature'][j]

            stock_train_df = train_df[stock][:index + i]

            # get the signal for tomorrow for that stock and add it to the list of signals
            signals.append(get_signal(models, stock_train_df, features, pred_feature))

            # computes if the strategy would have made money or not and how much
            strat.append( df_features[stock].Return[index + i] * signals[j] )
            returns.append(df_features[stock].Return[index + i])

        # get signals and how much to invest in each stock
        invest_amounts = get_investment_amount(signals, wealth, df_stocks)

        invest_amounts = np.array(invest_amounts)
        strat = np.array(strat)
        wealth += np.sum(invest_amounts * strat)

        train_df = df_features[:index + i]
    
    return wealth




In [180]:
# Creates test data

df_stocks = pd.DataFrame(['AAPL','BA'], columns = ['name'])
df_stocks['models'] = [[SVC(), DecisionTreeClassifier(), LogisticRegression(), MLPClassifier()], [SVC(), DecisionTreeClassifier(), LogisticRegression(), MLPClassifier()]]
df_stocks['pred_feature'] = ['Direction', 'Direction']
df_stocks['features'] = [['lag1','lag2'],['lag1','lag2']]

df_features = pd.DataFrame(
    {
        ('AAPL', 'lag1'): [0.1, 0.2, 0.3, 0.4, 0.5, 0.1],
        ('AAPL', 'lag2'): [0.4, 0.5, 0.6, 0.7, 0.8, 0.5],
        ('AAPL', 'Direction'): [1, -1, 1, -1, 1, 1],
        ('AAPL', 'Return'): [0.3,-0.1,0.2,-0.3,0.1,0.2],
        ('BA', 'lag1'): [0.2, 0.1, 0.3, 0.4, 0.5, 0.2],
        ('BA', 'lag2'): [0.5, 0.4, 0.6, 0.7, 0.8, 0.3],
        ('BA', 'Direction'): [-1, 1, -1, 1, -1, -1],
        ('BA', 'Return'): [-0.1,0.3,-0.2,0.3,-0.1,-0.2],
    },
    index=pd.date_range('2024-02-27', periods=6)
)
df_features.columns = pd.MultiIndex.from_tuples(df_features.columns)


trade(df_stocks, df_features, 50000)


/var/folders/2j/h6y_zlpd5sq1fhr8v9460g600000gn/T/ipykernel_4151/572227368.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  strat.append( df_features[stock].Return[index + i] * signals[j] )
/var/folders/2j/h6y_zlpd5sq1fhr8v9460g600000gn/T/ipykernel_4151/572227368.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  returns.append(df_features[stock].Return[index + i])
/Users/sebastianapelgren/code/computing-in-fin/Jupiter Notebooks/myenv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization h

np.float64(44000.0)

In [182]:
df_stocks

,name,models,pred_feature,features
0,AAPL,"[SVC(), DecisionTreeClassifier(), LogisticRegr...",Direction,"[lag1, lag2]"
1,BA,"[SVC(), DecisionTreeClassifier(), LogisticRegr...",Direction,"[lag1, lag2]"


In [ ]:
df_features

AAPL                         BA                      
           lag1 lag2 Direction Return lag1 lag2 Direction Return
2024-02-27  0.1  0.4         1    0.3  0.2  0.5        -1   -0.1
2024-02-28  0.2  0.5        -1   -0.1  0.1  0.4         1    0.3
2024-02-29  0.3  0.6         1    0.2  0.3  0.6        -1   -0.2
2024-03-01  0.4  0.7        -1   -0.3  0.4  0.7         1    0.3
2024-03-02  0.5  0.8         1    0.1  0.5  0.8        -1   -0.1
2024-03-03  0.1  0.5         1    0.2  0.2  0.3        -1   -0.2